In [0]:
%sql
CREATE TABLE IF NOT EXISTS soundhar_catalog.soundhar_schema.shipment_his_ingestion
USING DELTA
AS SELECT 
  shipment_id,
  first_name,
  last_name,
  age,
  role,
  city,
  updated_at  
FROM gcp_mysql_fc_we3.logistics.shipments1;

In [0]:
%sql
MERGE INTO soundhar_catalog.soundhar_schema.shipment_his_ingestion AS target
USING (
  SELECT * FROM gcp_mysql_fc_we3.logistics.shipments1
  WHERE updated_at > (SELECT COALESCE(MAX(updated_at), '1970-01-01') FROM soundhar_catalog.soundhar_schema.shipment_his_ingestion)
) AS source
ON target.shipment_id = source.shipment_id
WHEN MATCHED THEN
  UPDATE SET 
    target.first_name = source.first_name,
    target.last_name = source.last_name,
    target.age = source.age,
    target.role = source.role,
    target.updated_at = source.updated_at
WHEN NOT MATCHED THEN
  INSERT (shipment_id, first_name, last_name, age, role, updated_at)
  VALUES (source.shipment_id, source.first_name, source.last_name, source.age, source.role, source.updated_at);